# Evolution du prix du gazole

##### Auteurs : Mickaël Canhoto (L2 Informatique), Achille Gravouil (L1 Informatique)

*Notre projet consiste à étudier et traiter l'évolution du prix moyen par mois du gazole de 1992 à aujourd'hui.*

### Sommaire

 1. [Introduction](#Introduction)
 2. [Importation des modules](#Importation-des-modules)
 3. [Collecte des données](#Collecte-des-données)
 4. [Traitement des données](#Traitement-des-données)
 5. [Représentation graphique](#Représentation-graphique)
 6. [Conclusion](#Conclusion)

## Introduction

Suite à la situation tendue en Europe et la hausse des prix du gazole, nous avons décidé d'étudier les variations du prix du gazole sur les 30 dernières années. 
Cette étude a pour objectif d'étudier l'évolution du prix afin de creer une représentation graphique.

Le gazole est un carburant pour moteur issu du raffinage du pétrole. Il est principalement utilisé pour comme carburant pour les voitures ou camion. Mais aussi comme agent d'extraction, par exemple pour extraire du palladium

Nous essaierons de représenter le plus fidèlement et le plus explicitement, les données recueillis durant ce projet.
Nous verrons donc comment le prix à évolué et nous esssaierons d'anticiper le(s) futur(s) mouvement possible grâce à une fonction

Nous pourrons alors, tirer une conclusion sur le prix du gazole. Permettant ainsi à tous usagers d'automobiles, d'observer le prix de leurs consomations et réfléchir en tant que conclusion à d'autre alternative.

**Pour cela nous nous aiderons des informations recuellis par l'[INSEE](https://www.insee.fr/fr/statistiques/serie/000442588#Tableau)**

## Importation des modules


Pour importer, analyser et traiter les données recoltés, nous allons utiliser les modules suivants : 
1. Pandas (importation des données)
2. Bokeh & ipywidget & matplotlib (Création du graphique)
3. scipy (Calcul)
4. numpy (Modification des datas)

In [35]:
import plotly as pl
import numpy as np
import pandas as pd 

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.models import Div, RangeSlider, Spinner
from bokeh.layouts import layout
from bokeh.io import output_file, show

import ipywidgets as widgets

from scipy.stats import linregress

import warnings #Permet d'éviter le Warning 'openyxl's style'
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

## Collecte des données
 

 * Les données proviennent du site de l'[INSEE](https://www.insee.fr/fr/statistiques/serie/000442588#Tableau). Elles nous présentent le prix moyens mensuels de vente du litre de Gazole en métrople Française de 1992 a 2022.
 
 La bibliothèque panda permet de transformer des donneés d'un fichier excel en un tableau de ces données
 

In [36]:
data_base = pd.read_excel (r"./data/mesures.xlsx", skiprows=3) #importe les données depuis le fichier excel
data_base = np.flipud(data_base) #Inverse le tableau pour une meilleur visibilité

data_base = pd.DataFrame(data_base, columns=["Période", "Prix"])

with pd.option_context("display.max_rows", 10): #On affiche uniquement 10 lignes
    display(data_base)

,Période,Prix
0,1992-01,0.54
1,1992-02,0.54
2,1992-03,0.54
3,1992-04,0.53
4,1992-05,0.54
...,...,...
357,2021-10,1.56
358,2021-11,1.57
359,2021-12,1.54
360,2022-01,1.63


## Traitement des données

* ### Calcule de la moyenne par année
Pour une meilleur lisibilité, nous allons réduire le tableaux par année. Pour cela, nous allons crée une fonction permettant de calculer la moyenne de chaque année et en retourner un tableau

In [37]:
def moyenne_annee():
    
    # fonction "moyenne annee" :
    # Objectif : Calculer la moyenne des prix de chaque année
    # Valeur entrée : aucune
    # Valeur sortie : tableau des moyennes de chaque année
    
    #Initialisation
    data_moy_annee = pd.DataFrame({
        "Période": [],
        "Prix": []}
    )
    r=0
    #Première boucle parcours chaque année
    for i in range (0, 360, 12):
        x=0
        data_moy_annee.loc[r, "Période"] = data_base.loc[i, "Période"]
        
        #Deuxième boucle parcours chaque mois
        for a in range(i,i+12):
            x += data_base.loc[a, "Prix"]
            data_moy_annee.loc[r, "Prix"] = x/12
        r=r+1
        
    #L'année 2022 ne possède uniquement que 2 mois, on réalise alors une boucle appart
    x = 0
    for a in range(2):
        x += data_base.loc[(a + 360), "Prix"]

    
        data_moy_annee.loc[30, "Période"]= data_base.loc[360, "Période"]
        data_moy_annee.loc[30, "Prix"]= x/2
        
    data_moy_annee["Période"] = data_moy_annee["Période"].str[:4]
    return data_moy_annee

data_moy = moyenne_annee()
moyenne_annee().set_index("Période").transpose()

Période,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Prix,0.5375,0.560833,0.595,0.593333,0.6575,0.683333,0.645,0.685833,0.8475,0.799167,...,1.364167,1.301667,1.166667,1.111667,1.233333,1.4375,1.443333,1.273333,1.443333,1.675


* ### Evolution du prix
Nous allons maintenant s'interresser aux années où l'évolution du prix à été forte .

Pour cela nous calculons l'écart type de la moyenne par années puis nous gardons dans un tableau de toutes les années x ou l'écarte type/2 < x - x-1

In [38]:
def Ecart_type(tableau) :
    # fonction "Ecart Type" :
    # Objectif : Calculer l'écart type
    # Valeur entrée : tableau
    # Valeur sortie : écart type du tableau
    return tableau['Prix'].std()

def forte_evolution(tableau) :
    # fonction "Forte évolution" :
    # Objectif : Crée un tableau des anneés ou l'évolution du prix a été forte
    # Valeur entrée : tableau
    # Valeur sortie : tableau
    
    k = 0
    Ecart = pd.DataFrame ({
        "Période": [],
        "Prix": []}
    )
    # Ajoute  la ligne i de *tableau dans *Ecart si l'écart entre tableau[i-1] et tableau[i] est supérieur 
    # à *ecart_type/2 (i.e grosse variation)
    for i in range (1, len(tableau)-1) :
        if abs(tableau.loc[i, "Prix"] - tableau.loc[i-1, "Prix"]) > Ecart_type(tableau)/2 : # test si 
            Ecart.loc[k, "Période"] = tableau.loc[i, "Période"]
            Ecart.loc[k, "Prix"] = tableau.loc[i, "Prix"]
            k += 1
    return Ecart

evolution = forte_evolution(data_moy)
forte_evolution(data_moy).transpose()

,0,1,2,3,4,5
Période,2008,2009,2011,2018,2020,2021
Prix,1.2825,1.008333,1.345,1.4375,1.273333,1.443333


* ### Régression linéaire
La régression linéaire va nous permettre de prédire l'évolution du prix de l'essence.

Le principe de la régression va être de trouver une droite qui se rapproche le plus des points que nous avons.
Pour cela nous allons utilisé cette formule : F(x) = Slope * x + intercept

In [39]:
def reglineaire(x, y) :
    # fonction "Régression linéaire" :
    # Objectif : Calculer la pente et l'ordonnée
    # Valeur entrée : X, Y
    # Valeur sortie : Droite
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope * x + intercept
    
X = data_base["Période"].str[:4].astype(float)
Y = data_base["Prix"].astype(float)

* ### Préparation d'un tableau avec une colonne Année

Ce tableau nous permettra ensuite avec la librairie Ipywidget d'effectuer un graph comportant un menu déroulant.
Ce menu déroulant nous permettra de voir un graph montrant l'évolution du prix du gazole durant l'année de notre choix.


In [40]:
#je reimporte les data sauf l'année 2022 que je rajouterai manuellement après
data_tab = pd.read_excel (r"./data/mesures.xlsx", skiprows=3) 
data_tab = np.flipud(data_tab)
data_tab = pd.DataFrame(data_tab, columns=["Période", "Prix"])

annee = pd.DataFrame({"Année":[]})


r=0
for i in range (360):
    if (i % 12) == 0:
        r= data_base.loc[i, "Période"]
    annee.loc[i, "Année"] = r
    
    
annee["Année"] = annee["Année"].str[:4]
data_tab["Période"] = data_tab["Période"].str[5:]


#L'année 2022 ne possède uniquement que 2 mois, je fais donc une boucle à part
# et je rajoute 10 mois pour que mon display est toujours la même dimension
x = 1
for a in range(12):
    annee.loc[a + 360, "Année"] = 2022
    data_tab.loc[a + 360, "Période"] = str(a+1).zfill(2)
    x +=1    
    
    
data_tab["Année"]= annee
data_tab = data_tab[["Année", "Période", "Prix"]]
data_tab.set_index("Année").tail(20).transpose()

Année,2021,2021,2021,2021,2021,2021,2021,2021,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
Période,05,06,07,08,09,10,11,12,01,02,03,04,05,06,07,08,09,10,11,12
Prix,1.4,1.43,1.45,1.45,1.46,1.56,1.57,1.54,1.63,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Représentation graphique

Pour la représentation graphique, nous avons utilisé la librairie bokeh et holoviews ainsi que toutes les transformations de données ci-dessus

In [41]:
#Graphique de base

#creation des courbes
graph = figure(title="Evolution du prix du Gazole", 
              x_axis_label="Période (En année)", 
              y_axis_label="Prix (En Euro)",
              tooltips = [("Année ", "@x"), ("Prix ", "@y")],
              sizing_mode="stretch_width"
             )
graph.line(
    x = data_base["Période"].str[:4],
    y = data_base["Prix"], 
    legend_label="Prix par mois", 
    color='grey', 
    line_dash="5 5")

graph.line(
    x = data_moy["Période"], 
    y = data_moy['Prix'], 
    legend_label="Prix par années", 
    color='orange', 
    line_width=4
)

points = graph.circle(
    x = evolution["Période"], 
    y = evolution["Prix"], 
    legend_label="Forte évolution", 
    color='red', 
    size=10, 
    fill_alpha=0.5
)

#détais graphique
graph.legend.border_line_alpha = 0.8
graph.legend.location = "top_left"
graph.legend.label_text_font_style = "italic"
graph.line(X, reglineaire(X, Y), legend_label="Régression Linéaire", color='darkred')
#graph.background_fill_color = "beige" #Couleur du fond de la courbe
graph.toolbar.logo = None
graph.legend.click_policy="hide"


output_notebook()


Loading BokehJS ...

In [60]:
#Graphique par année

dropdown_annee=["Graph"]+[1992+i for i in range(31)]

#Dictionnaire de tous les mois
dict_mois = {1:"Janvier", 
             2:"Février", 
             3:"Mars", 
             4:"Avril",
             5:"Mai", 
             6:"Juin", 
             7:"Juillet", 
             8:"Août",
             9:"Septembre", 
             10:"Octobre", 
             11:"Novembre", 
             12:"Décembre"
            }

#Affichage par année
def aff_per_year ( Choisi = "Graph") :
    # fonction :"Affiche courbe par année" 
    # Objectif : Crée une courbe du prix sur l'année demander
    # Valeur entrée : dictionnaire dropdown_annee (valeur de ce dictionnaire return par le widgets.interact)
    # Valeur sortie : Graphique
    
    #init dataframe z
    if (Choisi=="Graph"):
        show(graph)
    else:
        z = pd.DataFrame({
                     "Année":[],
                     "Période":[],
                     "Prix":[]
                    })

        #rempli la colonne Année avec Choisi 
        for i in range(12):
            z.loc[i, "Année"] = str(Choisi).zfill(0)
            
        
        #   Le fichier commence à l'année 1992 je cherche donc la différence entre l'année 
        #   recherché et je multiplie par 12 pour trouver le 1er rang de l'année que je veux
        #   Je fais ensuite une boucle pour rajouter le prix dans mon dataframe et je rempli la colonne Période
        diff = Choisi - 1992

        for a in range(12):
            z.loc[a, "Prix"] = data_tab.loc[(diff*12) + a, "Prix"]
            z.loc[a, "Période"] = list(dict_mois.values())[a]
            
        #   Je crée ensuite le graph avec bokeh qui sera display
        graph_per_year = figure(
                  title=f"Evolution du prix du Gazole sur l'année {Choisi}", 
                  x_range = z["Période"],
                  x_axis_label="Période (En mois)", 
                  y_axis_label="Prix (En Euro)",
                  tooltips = [("Prix ", "@y"), ("Mois ", "@x")],
                  sizing_mode="stretch_width"
                 )
        graph_per_year.line(
                x = z["Période"],
                y = z["Prix"], 
                legend_label="Prix par mois", 
                color='orange', 
                line_width= 3
                )
        graph_per_year.legend.border_line_alpha = 0.8
        graph_per_year.legend.location = "top_left"
        graph_per_year.legend.label_text_font_style = "italic"
        #graph_per_year.background_fill_color = "beige"
        graph_per_year.toolbar.logo = None

        show(graph_per_year)
    

courbe = widgets.interact(aff_per_year, Choisi = dropdown_annee)


interactive(children=(Dropdown(description='Choisi', options=('Graph', 1992, 1993, 1994, 1995, 1996, 1997, 199…

## Conclusion

Avec la courbe, on remarque rapidement que le prix du gazole augmente chaque année passant de 0.532€ en 1992 à 1.720 début 2022.

Cette montée de prix peut s'expliquer de plusieurs manières :

- La montée du cours du prix du pétrole à cause d'une demande en pétrole qui augmente au fil des anneés. De plus, la production peut être freinée par des conflits géopolitiques telles que le conflit Ukraine Russie. Ou aussi l'OPEP (Organisation des pays exportateurs de pétrole) qui on la possibilité de faire varier le prix du cours.

- L'augmentation des taxes qui ont augmenté de 40% de 2014 à 2021 sur le litre.

- La quantité limitée de la ressource, augmentant alors sa rareté au fil de son utilisation.

Voici ci-dessous une fonction vous permettant de calculer le prix d'un plein en fonction de l'année et le litrage choisi. Il faut modifier la variable "Litre" et "Année" à votre guise

In [43]:
def calcul_plein(Litre, Année, Mois) :
    #Prend en entrée le litre, année et le mois choisi pour faire ce calcul :
    # Prix (mois choisi) * Litre
    Prix = data_base.loc[(((Année-1992)*12)+Mois), "Prix"]
    return Prix * Litre


In [45]:
Litre = input("Entrez le nombre de litre : ")
l = float(Litre) #Convertion en float
b = "" #initialise b

check = True
while check :
    Année = input("Et en quel année (entre 1992 et 2022) : ")
    a = float(Année)
    if (a>=1992) and (a<=2022):
        check = False
    else:
        print("Veuillez choisir entre 1992 et 2022")
        
check = True
#Vérifications de 
while check :
    Mois = input("Et en quel mois (Janvier, Février, ...) : ")
    for cle, valeur in dict_mois.items():
        if (valeur == Mois):
            b = cle
            check = False
    if not b : print("Vérifiez l'orthographe")
    
plein = calcul_plein(l, a, b)
print("\nEn", Mois, Année,", pour", Litre, "litre, un plein vous aurait coûté", format(plein,".2f"),"€ !")

Entrez le nombre de litre : 55
Et en quel année (entre 1992 et 2022) : 2008
Et en quel mois (Janvier, Février, ...) : Décembre
Vérifiez l'orthographe
Et en quel mois (Janvier, Février, ...) : Decembre

En Decembre 2008 , pour 55 litre, un plein vous aurait coûté 54.45 € !


### Alternatives 
Le gazole étant une ressource limitée, il faut alors étudier d'autres alternatives. Les premières idées ont été de s'intéresser sur les autres hydrocarbures que le pétrole :

- le gaz de pétrole liquéfié (GPL)

- les procédés Fischer-Tropsch CTL, GTL et BTL

- Le gaz naturel pour véhicule (GNV) et le biogaz

Malheureusement, ces ressources aussi restent limité dans le temps. On peut alors regarder des énergies plus renouvelables :

- Les biocarburants à base d'alcool (bioéthanol) ou d'huile (biodiesel)

- Le dihydrogène (H2)

- carburant à base de colza

- oxydoréduction (eau de mer) comme la quant e-sportlimousine

Mais l'alternative la plus efficace pour le moment reste l'électrique. L'électricité peut être produite par plusieurs manières comme par exemple le nucléaire ou le solaire et est beaucoup moins cher que le gazole.

Cependant, cette technologie reste assez récente et les batteries actuelles fonctionnent sur le lithium qui est limité et pollue énormément.

L'hybride quant à elle peut être un bon équilibre actuel entre le gazole et l'électricité.

Pour conclure, la montée constante du prix du gazole ouvre d'énorme question sur nos moyens de consommations et sera un thème majeur dans les futurs années. En effet, une grosse partie des industries et des citoyens utilisent quotidiennement des ressources à quantité limitées.